In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Load the graph data
# Create a graph from the data
G = nx.from_pandas_edgelist(df, source=0, target=1)

# Preprocessing
# Remove any nodes with missing values (if applicable)
G.remove_nodes_from(list(nx.isolates(G)))

# Feature Engineering
# Extract local features such as node degrees and centrality measures
node_degrees = dict(G.degree())
node_closeness_centrality = nx.closeness_centrality(G)
node_betweenness_centrality = nx.betweenness_centrality(G)

# Prepare feature data
feature_data = pd.DataFrame({'Degree': node_degrees,
                             'Closeness Centrality': node_closeness_centrality,
                             'Betweenness Centrality': node_betweenness_centrality}).values

# Standardize feature data
scaler = StandardScaler()
feature_data = scaler.fit_transform(feature_data)

# Elbow Method for determining the optimal number of clusters
k_values = range(2, 11)  # Range of k values to try
inertia_values = []
silhouette_scores = []
calinski_harabasz_scores = []
davies_bouldin_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k)
    cluster_labels = kmeans.fit_predict(feature_data)
    inertia_values.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(feature_data, cluster_labels))
    calinski_harabasz_scores.append(calinski_harabasz_score(feature_data, cluster_labels))
    davies_bouldin_scores.append(davies_bouldin_score(feature_data, cluster_labels))

# Plotting the Elbow Curve
plt.figure(figsize=(10, 6))
plt.plot(k_values, inertia_values, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method: Inertia vs Number of Clusters')
plt.show()

# Plotting the Silhouette Score
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

# Plotting the Calinski-Harabasz Score
plt.figure(figsize=(10, 6))
plt.plot(k_values, calinski_harabasz_scores, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Score vs Number of Clusters')
plt.show()

# Plotting the Davies-Bouldin Score
plt.figure(figsize=(10, 6))
plt.plot(k_values, davies_bouldin_scores, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score vs Number of Clusters')
plt.show()

# Clustering with the optimal number of clusters
optimal_k = k_values[np.argmin(inertia_values)]
kmeans = KMeans(n_clusters=optimal_k)
cluster_labels = kmeans.fit_predict(feature_data)

# Evaluation
silhouette = silhouette_score(feature_data, cluster_labels)
calinski_harabasz = calinski_harabasz_score(feature_data, cluster_labels)
davies_bouldin = davies_bouldin_score(feature_data, cluster_labels)

print(f"Optimal Number of Clusters (k): {optimal_k}")
print(f"Silhouette Score: {silhouette}")
print(f"Calinski-Harabasz Score: {calinski_harabasz}")
print(f"Davies-Bouldin Score: {davies_bouldin}")

plt.figure(figsize=(10, 10)) 
pos = nx.spring_layout(G)
for i in range(optimal_k):
    nodes = [node for node, label in zip(G.nodes(), cluster_labels) if label == i]
    nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=f"C{i}", node_size=50, label=f"Cluster {i}")
nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=0.5)
nx.draw_networkx_labels(G, pos, labels={}, font_size=8)  # Remove node labels

plt.title("Graph Representation with Clusters")  # Add a title
plt.legend()  # Add a legend
plt.axis('off')  # Remove the axis

plt.show()
